# Ενδιάμεση Εργασία:

Θα γράψουμε ένα πρόγραμμα στη γλώσσα python, με τη βοήθεια της βιβλιοθήκης pytorch που θα συγκρίνει την απόδοση του κατηγοριοποιητή πλησιέστερου γείτονα με 1 και 3 πλησιέστερους γείτονες, με τον κατηγοριοποιητή πλησιέστερου κέντρου μιας βάσης δεδομένων. Επιλέχθηκε η χρήση της γλώσσας python, και συγκεκριμένα της βιβλιοθήκης pytorch, επειδή τη σήμερων ημέρα αυτές είναι το industry standard στον κλάδο του machine learning και ήταν ευκαιρία να τις μάθουμε. 

Οι κατηγοριοποιητές μας δοκιμάστηκαν σε διάφορες βάσεις δεδομένων, επιλέχτηκε όμως τελικά να παρουσιαστεί η δοκιμή τους στη βάση δεδομένων cifar-10: https://www.cs.toronto.edu/~kriz/cifar.html επειδή αποτελείται από 60000 32x32 έγχρωμες φωτογραφίες 10 κλάσεων, με ίσο αριθμό δειγμάτων ανα κλάση. Αυτό είναι πολύ σημαντικό, επειδή βοηθάει ώστε να μην έχουμε bias μιας κλάσης σε σχέση με μια άλλη, επειδή έχουμε άνισο αριθμό δεδομένων. Επίσης, η βάση αυτή έχει ήδη ξεχωρίσει 50000 εικόνες για training και 10000 εικόνες για testing του μοντέλου μας.

## GPU Compatibility

Στη δική μας μηχανή υπάρχει GPU της εταιρείας NVIDIA και το CUDA είναι ήδη εγκατεστημένο. Παρ'όλα αυτά, αυτό μπορεί να μην ισχύει για όλες τις συσκευές. Για αυτόν τον λόγο, γράψαμε τον παρακάτω κώδικα ώστε να τρέχει σε CUDA, αν αυτό υποστηρίζεται, ενώ σε κάθε άλλη περίπτωση να τρέχει στην CPU.


In [182]:
import torch
# Set device type
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Στο πείραμά μας, θα αναφέρουμε τον χρόνο που πήρε κάθε κομμάτι κώδικα για να τρέξει στη δική μας μηχανή. Για αυτόν τον λόγο, θα απαριθμήσουμε παρακάτω τα specs του υπολογιστή ώστε να είναι η ανάλυσή μας πλήρης:

- Motherboard: ASRock B650M Pro RS Motherboard Micro ATX με AMD AM5 Socket
- RAM: Adata 32GB DDR5 RAM με Ταχύτητα 6400
- SSD: Western Digital SN770 SSD 2TB M.2 NVMe PCI Express 4.0
- CPU: AMD Ryzen 7 7800X3D 8-Core Processor 4.20 GHz
- GPU: NVIDIA GeForce GTX 1060 3GB



## K-nearest neighbors
Αρχικά, θα γράψουμε τη συνάρτηση που βρίσκει τους k-nearest neighbors από ένα tensor με βάση κάποια training data. Ιδανικά θέλουμε να γράψουμε μια συνάρτηση στην οποία μπορούμε να δώσουμε έναν πίνακα $ Q \varepsilon \mathbb{R}^{nxm} $, όπου m είναι η διάσταση του χώρου (ανάλογα σε ποιόν χώρο ψάχνουμε), ενώ n είναι ο αριθμός όλων των σημείων των οποίων την απόσταση ψάχνουμε. Στην περίπτωσή μας, το n είναι πάντα 10000 αφού αυτό είναι το test data μας. Ο λόγος που ψάχνουμε για όλα τα σημεία με μόνο μια κλήση της συνάρτησης είναι επειδή ο συγκεκριμένος υπολογισμός παίρνει χρόνο αλλά είναι παραλληλοποιήσιμος. Παρατηρήθηκε μεγάλη αύξηση της απόδοσης του προγράμματος όταν η συνάρτηση γράφηκε όπως εξηγήσαμε. Επίσης, για να είναι το πρόγραμμα παραλληλοποιήσιμο χρησιμοποιήσαμε τις έτοιμες συναρτήσεις της pytorch που τρέχουν στην GPU. Συνεπώς έχουμε:

In [183]:
def knn(test_data, training_data, labels, k=3):
    # Calculate Euclidean distances of each test point with each training point
    distances = torch.cdist(test_data, training_data)

    # Get the indices of the k nearest neighbors for each test point
    knn_indices = torch.topk(distances, k, largest=False).indices

    #Use the indeces to get the correct labels
    knn_labels = labels[knn_indices] 

    # Get the most common label for each test point
    predictions = torch.mode(knn_labels, dim=1).values
    
    ##NOTE: In case of tie between label frequency, torch.mode() returns the smallest value out of the ones that tied

    return predictions

Επιλέξαμε ως προεπιλογή η συνάρτηση να τρέχει την version του 3-Nearest-Neighbors αν το k δεν δηλωθεί ρητά. Μπορούμε φυσικά να τρέξουμε και την 1-Nearest-Neighbor καλώντας την συνάρτηση για k = 1

# Nearest Centroid

Η μέθοδος του Nearest Centroid ανάγεται στην εύρεση των Centroids $\mathbf{C}_i, \;\; i = 1,2,...,10$ των data μας για κάθε κλάση: $$ \mathbf{C}_i = [y_1 , y_2, ...,y_n], \;\; y_i = \dfrac{1}{1000}\sum_{j = 1}^{1000} x_{ij} $$

Όπου αν τα training data είναι ένας πίνακας $\mathbf{T} \varepsilon \mathbb{R}^{qxw},\; q = 1000, \; w = 10$ ο οποίος έχει τα training data για κάθε κλάση σε μοναδικές και διαφορετικές μεταξύ τους στήλες, τότε: $$ x_{ij} = \mathbf{T}_{i,j}$$

Παρακάτω υλοποιούμε τον κώδικα υπολογισμού των γεωμετρικών μέσων.

In [184]:
import numpy as np
def compute_centroids(data_points,labels):
    centroids = []
    for i in range(10):
        #mask all data that is in the i-th class and store its mean
        centroids.append(data_points[labels == i].mean(axis=0))
    return np.array(centroids)

Με την εύρεση των $\mathbf{C}_i$, μετά μπορούμε να χρησιμοποιήσουμε τον ήδη υλοποιημένο αλγόριθμο του knn με k = 1 και δίνοντάς του ως training data τη λίστα των centroids, ώστε να βρούμε τον κοντινότερο γεωμετρικό μέσο για τα test data. 

## Κύριο πείραμα

Τώρα που έχουμε όλες τις συναρτήσεις μας έτοιμες, μπορούμε να εκτελέσουμε το πείραμά μας. Αρχικά, ενώ έχουμε κατεβάσει την cifar-10 βάση δεδομένων, συμβουλευόμαστε την ιστοσελίδα της βάσης για το πώς να ανακτήσουμε σωστά τα δεδομένα. Εκεί, μας δίνεται η παρακάτω συνάρτηση.

In [185]:
import pickle
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

Διαβάζοντας επιπλέον τις πληροφορίες που μας δίνονται, αναφέρουμε ότι η βάση δεδομένων είναι χωρισμένη σε 5 data batches και 1 test batch. Τα data batches αποτελούνται το καθένα απο 1000 διανύσματα $3 \cdot 1024 = 3072$ διαστάσεων το καθένα, που αποτελούν τις τιμές των πίξελ της εικόνας για τα RGB κανάλια αντίστοιχα.

Επιλέγουμε αρχικά να εκτελέσουμε τους αλγορίθμους μας με βάση τα διανύσματα αυτά, δηλαδή με βάση τις τιμές των pixels της κάθε εικόνας και χωρίς κάποια προεπεξεργασία. Για αυτόν τον λόγο, δημιουργήσαμε από τα 5 data batches ένα διάνυσμα train_data καθώς και train_labels για τα labels τους, καταλήγοντας δηλαδή με 50000 διανύσματα.

In [186]:
batches = []
for i in range(5):
    #add to the batches list, the arrays of each batch
    batches.append(unpickle(f"cifar-10-batches-py/data_batch_{i+1}"))
#stack the arrays vertically, resulting in a 50000x3072 matrix
train_data = np.vstack((batches[0][b'data'],batches[1][b'data'],batches[2][b'data'],batches[3][b'data'],batches[4][b'data']),dtype=np.uint8)
#stack the labels horizontally, resulting in a 1x50000 matrix
train_labels = np.hstack((batches[0][b'labels'],batches[1][b'labels'],batches[2][b'labels'],batches[3][b'labels'],batches[4][b'labels']),dtype=np.int64)
#prepare the test data in array form
test_batch = unpickle("cifar-10-batches-py/test_batch")
test_data = test_batch[b'data']
test_labels = test_batch[b'labels']

Αρκεί τώρα να καλέσουμε την συνάρτησή μας με τα ορίσματα test_data, train_data, train_labels η οποία θα μας επιστρέψει τον πίνακα με τα labels που κάνει predict ο knn αλγόριθμος για τα test_data. Η γραμμή κώδικα αυτή βγήκε μακριά επειδή ταυτόχρονα κάνουμε μετατροπή των arrays σε torch tensors οι οποίοι είναι συμβατοί με τις συναρτήσεις μας. Αναφέρουμε ρητά το device και το dtype καθενός για να αποφύγουμε τυχόν λάθη που σχετίζονται με τύπους δεδομένων και ακρίβεια.

In [187]:
predictions3NN = knn(torch.tensor(test_data,device=device,dtype=torch.float64),torch.tensor(train_data,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64))


Στη μηχανή μας, ο αλγόριθμος τερματίζει σε 3.1s

Αφού λοιπόν έχουμε τα πραγματικά test labels καθώς και τα predictions, αφαιρώντας τα μεταξύ τους δημιουργούμε ένα διάνυσμα το οποίο έχει μηδενικό εκεί που το prediction είναι σωστό, και κάποιον άλλον αριθμό εκεί που είναι λανθασμένο. Συνεπώς, μετρώντας τα μηδενικά μπορούμε να βρούμε το success rate του αλγόριθμου σε ποσοστό.

In [188]:
def print_success_prc(predictions,test_labels, return_value = False):
    diff = predictions-test_labels
    success_cntr = 0
    for value in diff:
        if abs(value.item()) < 0.1: #we assume it's the same value since the values differ by at least 1
            success_cntr += 1
    success_percent = float(success_cntr)/len(diff) * 100
    #print(f"success percentage: %.12f running with {device}\n" % success_percent)
    if(return_value):
        return success_percent
    else:
        print(f"success percentage: %.2f running with {device}\n" %success_percent)

print_success_prc(predictions3NN,torch.tensor(test_labels,device=device,dtype=torch.float32))

success percentage: 33.03 running with cuda



Παραπάνω τρέξαμε το πρόγραμμά μας στην περίπτωση του 3-Nearest-Neighbor αλγορίθμου. Σύντομα, παρουσιάζουμε κώδικα και για την περίπτωση 1-Nearest-Neighbor καθώς και του Nearest-Centroid.

In [189]:
#1-Nearest-Neighbor
predictions1NN = knn(torch.tensor(test_data,device=device,dtype=torch.float64),torch.tensor(train_data,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=1)
print_success_prc(predictions1NN,torch.tensor(test_labels,device=device,dtype=torch.float32))

success percentage: 35.39 running with cuda



In [190]:
#Nearest Centroid
centroids = compute_centroids(train_data,train_labels)
predictionsNC = knn(torch.tensor(test_data,device=device,dtype=torch.float64),torch.tensor(centroids,device=device,dtype=torch.float64),torch.tensor([0,1,2,3,4,5,6,7,8,9],device=device,dtype=torch.float64),k=1)
print_success_prc(predictionsNC,torch.tensor(test_labels,device=device,dtype=torch.float32))

success percentage: 27.74 running with cuda



## Επαλήθευση

Η επαλήθευση του κώδικά μας είναι εύκολη επειδή υπάρχουν έτοιμες βιβλιοθήκες που έχουν υλοποιήσει τον KNN αλγόριθμο, καθώς και τον Nearest Centroid και γνωρίζουμε ότι τα αποτελέσματα που μας δίνουν είναι σωστά.

Μια τέτοια βιβλιοθήκη είναι η scikit-learn την οποία θα χρησιμοποιήσουμε παρακάτω για να επαληθεύσουμε τα αποτελέσματά μας.

In [191]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestCentroid

In [192]:
# 3-Nearest-Neighbors
knn_scikit = KNeighborsClassifier(n_neighbors=3)
knn_scikit.fit(train_data,train_labels)
predictions3ΝΝ_scikit = knn_scikit.predict(test_data)
accuracy = accuracy_score(test_labels,predictions3ΝΝ_scikit)
print("success percentage:",accuracy*100)

success percentage: 33.03


In [193]:
# 1-Nearest-Neighbors
knn_scikit = KNeighborsClassifier(n_neighbors=1)
knn_scikit.fit(train_data,train_labels)
predictions1ΝΝ_scikit = knn_scikit.predict(test_data)
accuracy = accuracy_score(test_labels,predictions1ΝΝ_scikit)
print("success percentage:",accuracy*100)

success percentage: 35.39


In [194]:
# Nearest Centroid
cent = NearestCentroid()
cent.fit(train_data,train_labels)
predictionsNC_scikit = cent.predict(test_data)
accuracy = accuracy_score(test_labels,predictionsNC_scikit)
print("success percentage:",accuracy*100)

success percentage: 27.74


Αφού τα αποτελέσματά μας είναι ίδια, αν εξαιρέσουμε την παραπάνω λεπτομέρεια, είμαστε σίγουροι πια για την ορθότητα του κώδικά μας και το πείραμα έχει τελειώσει

## Freestyle

Παρατηρούμε ότι τα ποσοστά επιτυχίας και για τους 3 αλγορίθμους δεν είναι πολύ υψηλά. Αυτό είναι αναμενόμενο, μιας και οι αλγόριθμοί μας είναι ακατάλληλοι για εικόνες και γενικά δεν πετυχαίνουν πολυ υψηλά ποσοστά. Παραπάνω χρησιμοποιήσαμε την πιο απλή μέθοδο, δηλαδή δημιουργήσαμε ένα διάνυσμα των 3072 διαστάσεων για κάθε εικόνα, που αντιπροσωπεύει τα RGB values για κάθε pixel, και το δώσαμε στους αλγορίθμους μας χωρίς κάποια επεξεργασία. Με αυτόν τον τρόπο, δεν χάνεται απολύτως καθόλου πληροφορία μέσω επεξεργασίας, αλλά δεν είναι όλη η πληροφορία χρήσιμη, ούτε και διαφωτιστική. Υποπτευόμαστε ότι θα μπορούσαμε ίσως να αυξήσουμε τα ποσοστά επιτυχίας χρησιμοποιώντας μεθόδους ψηφιακής επεξεργασίας εικόνας. Ευτυχώς, η python έχει βιβλιοθήκες όπως η opencv που έχουν ήδη υλοποιημένες τις μεθόδους αυτές, μιας και η ψηφιακή επεξεργασία εικόνας είναι ένα εντελώς άλλο πεδίο γνώσης με ευρύ φάσμα, από το οποίο κατέχουμε μόνο κάποιες βασικές έννοιες.

- Η πρώτη ιδέα είναι η δημιουργία ενός ιστογράμματος, ώστε να κρατήσουμε μόνο βασικές πληροφορίες της εικόνας. Θα δημιουργήσουμε 3 ιστογράμματα για κάθε εικόνα, ένα για κάθε κανάλι, και να συγκρίνουμε τα δεδομένα αυτών με τους αλγορίθμους μας στο τέλος

- Μια δεύτερη ιδέα είναι η χρήση του Histogram of Oriented Graphs (HOG) για την επεξεργασία της πληροφορίας και εκτέλεση των αλγορίθμων

- Κάποιος αλγόριθμος edge detection επίσης ίσως μας δώσει χρήσιμα δεδομένα για την εικόνα που να μας αυξήσουν το ποσοστό επιτυχίας

- Μερικές πηγές πρότειναν τη μέθοδο LBP (Local Binary Pattern). Αν και δεν πιστεύουμε ότι θα βοηθήσει σύμφωνα με τον τρόπο με τον οποίο λειτουργεί, δεν χάνουμε κάτι να τη δοκιμάσουμε.




In [195]:
from skimage import color
from skimage.feature import local_binary_pattern
from skimage.feature import hog
def lbp_features(image):
    gray_image = color.rgb2gray(image)
    gray_image = (gray_image * 255).astype(np.uint8)

    # Parameters for LBP
    radius = 1
    n_points = 8 * radius

    lbp = local_binary_pattern(gray_image, n_points, radius, method="uniform")
    return lbp

def normalize(array):
    return (array - np.min(array,axis=0)/(np.max(array,axis=0)-np.min(array,axis=0)))


In [196]:
import cv2

#Reshape the train data vector so that each image is 3 (RGB) x 32 x 32
images_sep = train_data.reshape(50000,3,1024).reshape(50000,3,32,32)

#initialize all vectors for storing method info
hist_data = np.zeros((50000,3,256),np.float32)
hog_features = np.zeros((50000,324),np.float32)
edge_features = np.zeros((50000,1024),np.float32)
img_features = np.zeros((50000,1024),np.float32)

for index1,image in enumerate(images_sep):
    for index2,color_channel in enumerate(image):
        #calculate histogram for each RGB
        hist_data[index1,index2] = cv2.calcHist([color_channel],[0],None,[256],[0,256]).reshape(256)
    hog_features[index1], hog_image = hog(image.transpose(1,2,0),pixels_per_cell=(8, 8),cells_per_block=(2, 2),visualize=True,channel_axis=-1,transform_sqrt=True)
    gray_img = cv2.cvtColor(image.transpose(1,2,0),cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_img, (5, 5), 1)
    # Perform Canny edge detection
    edges = cv2.Canny(blurred_image, threshold1=100, threshold2=200)
    edge_features[index1] = edges.flatten().reshape(1024)
    img_features[index1] = lbp_features(image.transpose(1,2,0)).flatten()
hist_data = hist_data.reshape(50000,768)

#same process for the test data
test_images_sep = test_data.reshape(10000,3,1024).reshape(10000,3,32,32)
hist_data_t = np.zeros((10000,3,256),np.float32)
hog_features_t = np.zeros((10000,324),np.float32)
edge_features_t = np.zeros((10000,1024),np.float32)
img_features_t = np.zeros((10000,1024),np.float32)

for index1,image in enumerate(test_images_sep):
    for index2,color_channel in enumerate(image):
        hist_data_t[index1,index2] = cv2.calcHist([color_channel],[0],None,[256],[0,256]).reshape(256)
    hog_features_t[index1], hog_image = hog(image.transpose(1,2,0),pixels_per_cell=(8, 8),cells_per_block=(2, 2),visualize=True,channel_axis=-1,transform_sqrt=True)
    gray_img = cv2.cvtColor(image.transpose(1,2,0),cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_img, (5, 5), 1)
    # Perform Canny edge detection
    edges = cv2.Canny(blurred_image, threshold1=100, threshold2=200)
    edge_features_t[index1] = edges.flatten().reshape(1024)
    img_features_t[index1] = lbp_features(image.transpose(1,2,0)).flatten()
hist_data_t = hist_data_t.reshape(10000,768)

Τώρα που έχουμε όλα τα features που περιγράφηκαν πάνω, μπορούμε να χρησιμοποιήσουμε τους αλγορίθμους μας για να δούμε ποιό μας δίνει το καλύτερο αποτέλεσμα.

In [197]:
hist_predictions_1NN = knn(torch.tensor(hist_data_t,device=device,dtype=torch.float64),torch.tensor(hist_data,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=1)
hog_predictions_1NN = knn(torch.tensor(hog_features_t,device=device,dtype=torch.float64),torch.tensor(hog_features,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=1)
edge_predictions_1NN = knn(torch.tensor(edge_features_t,device=device,dtype=torch.float64),torch.tensor(edge_features,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=1)
img_predictions_1NN = knn(torch.tensor(img_features_t,device=device,dtype=torch.float64),torch.tensor(img_features,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=1)

hist_predictions_3NN = knn(torch.tensor(hist_data_t,device=device,dtype=torch.float64),torch.tensor(hist_data,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=3)
hog_predictions_3NN = knn(torch.tensor(hog_features_t,device=device,dtype=torch.float64),torch.tensor(hog_features,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=3)
edge_predictions_3NN = knn(torch.tensor(edge_features_t,device=device,dtype=torch.float64),torch.tensor(edge_features,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=3)
img_predictions_3NN = knn(torch.tensor(img_features_t,device=device,dtype=torch.float64),torch.tensor(img_features,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=3)

hist_centroids = compute_centroids(hist_data,train_labels)
hog_centroids = compute_centroids(hog_features,train_labels)
edge_centroids = compute_centroids(edge_features,train_labels)
img_centroids = compute_centroids(img_features,train_labels)

hist_predictions_NC = knn(torch.tensor(hist_data_t,device=device,dtype=torch.float64),torch.tensor(hist_data,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=1)
hog_predictions_NC = knn(torch.tensor(hog_features_t,device=device,dtype=torch.float64),torch.tensor(hog_features,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=1)
edge_predictions_NC = knn(torch.tensor(edge_features_t,device=device,dtype=torch.float64),torch.tensor(edge_features,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=1)
img_predictions_NC = knn(torch.tensor(img_features_t,device=device,dtype=torch.float64),torch.tensor(img_features,device=device,dtype=torch.float64),torch.tensor(train_labels,device=device,dtype=torch.float64),k=1)

Ο παραπάνω κώδικας τρέχει σε κάτω απο 3 λεπτά (2m 42.5s) στην μηχανή μας χάρη στην παραλληλοποίηση!

In [198]:
print("\t \t PIXL \t\t \t HIST \t \t \t HOG \t \t \t EDGE \t \t \t IMG\n")
for i in range(5):
    print("\t  1-NN \t 3-NN \t NC",end = "")
print("\n")

pixl_perc_1NN = print_success_prc(predictions1NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
pixl_perc_3NN = print_success_prc(predictions3NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
pixl_perc_NC = print_success_prc(predictionsNC,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)

print("\t %.2f\t %.2f \t %.2f"% (pixl_perc_1NN,pixl_perc_3NN,pixl_perc_NC), end="")

hist_perc_1NN = print_success_prc(hist_predictions_1NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
hist_perc_3NN = print_success_prc(hist_predictions_3NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
hist_perc_NC = print_success_prc(hist_predictions_NC,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)

print("\t %.2f\t %.2f \t %.2f"% (hist_perc_1NN,hist_perc_3NN,hist_perc_NC), end="")

hog_perc_1NN = print_success_prc(hog_predictions_1NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
hog_perc_3NN = print_success_prc(hog_predictions_3NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
hog_perc_NC = print_success_prc(hog_predictions_NC,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)

print("\t %.2f\t %.2f \t %.2f"% (hog_perc_1NN,hog_perc_3NN,hog_perc_NC), end="")

edge_perc_1NN = print_success_prc(edge_predictions_1NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
edge_perc_3NN = print_success_prc(edge_predictions_3NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
edge_perc_NC = print_success_prc(edge_predictions_NC,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)

print("\t %.2f\t %.2f \t %.2f"% (edge_perc_1NN,edge_perc_3NN,edge_perc_NC), end="")

img_perc_1NN = print_success_prc(img_predictions_1NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
img_perc_3NN = print_success_prc(img_predictions_3NN,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)
img_perc_NC = print_success_prc(img_predictions_NC,torch.tensor(test_labels,device=device,dtype=torch.float32),return_value=True)

print("\t %.2f\t %.2f \t %.2f"% (img_perc_1NN,img_perc_3NN,img_perc_NC), end="")


	 	 PIXL 		 	 HIST 	 	 	 HOG 	 	 	 EDGE 	 	 	 IMG

	  1-NN 	 3-NN 	 NC	  1-NN 	 3-NN 	 NC	  1-NN 	 3-NN 	 NC	  1-NN 	 3-NN 	 NC	  1-NN 	 3-NN 	 NC

	 35.39	 33.03 	 27.74	 26.72	 26.43 	 26.72	 49.02	 49.86 	 49.02	 16.70	 15.30 	 16.70	 11.55	 10.42 	 11.55

Παρατηρούμε ότι δεν πετύχαμε και πολλά. Η χρήση των δεδομένων που ονομάσαμε IMG ήταν η χειρότερη, με περίπου 10% success rate που είναι σαν να επιστρέφουμε μια random κλάση για κάθε εικόνα.

Παρ΄όλα αυτά, φαίνεται η μέθοδος HOG να μας επέστρεψε τα καλύτερα δεδομένα με απόδοση περίπου 49%. Συμπαιρένουμε ότι η μέθοδος HOG είναι πολύ καλή για την λύση τέτοιων προβλημάτων, ακόμη και με τον απλό αυτό αλγόριθμο. Το 49% όμως σίγουρα δεν είναι μια πιθανότητα που είναι ικανοποιητική.

Μεταξύ των αλγορίθμων, απρόσμενα ο 1-NN φαίνεται στην περίπτωσή μας καλύτερος από τους υπόλοιπους. Γνωρίζουμε όμως ότι γενικά αυτό δεν ισχύει και απλά τυχαίνει να έχουμε αυτά τα αποτελέσματα με τα συγκεκριμένα δεδομένα. Αυτό που φέρει την μεγαλύτερη έκπληξη είναι ότι η μέθοδος Nearest Centroid δεν είναι όσο κακή περιμέναμε και διαφέρει συνήθως στο success rate κατά λιγότερο από 2%. Και πάλι, το φαινόμενο είναι τυχαιοκρατικό, όμως βασιζόμενοι στο πόσο λιγότερη υπολογιστική ισχύ καταναλώνει η μέθοδος και πόσο μεγαλύτερη ταχύτητα έχει, το αποτέλεσμά μας δεν φάνηκε καθόλου άσχημο.

Είναι εύκολο να συνδιαστούν τα δεδομένα κάθε μίας μεθόδου προεπεξεργασίας εικόνας που δείξαμε σε ένα διάνυσμα και να τρέξουμε τους αλγορίθμους μας έχοντας πχ (pixl,hog) χώρο αντί για (pixl) και (hog) ξεχωριστά. Η αναφερόμενη μέθοδος είναι απλή και δεν θα αναλυθεί, παρ'όλα αυτά δοκιμάστηκε, και γενικά φάνηκε να μειώνει την επιτυχία των μεθόδων αντί να την αυξάνει. Επίσης, αυξάνεται το μέγεθος του διανυσματικού χώρου και ο κώδικας γίνεται ακόμη πιο αργός. Σε κάθε περίπτωση, δεν θα το συνιστούσαμε. Φαίνεται πως γενικά οι αλγόριθμοί μας δεν δουλεύουν καλά σε εικόνες, οπότε η έξυπνη λύση θα ήταν απλά η εύρεση ενός καλύτερου αλγορίθμου.

Το πείραμά μας τελειώνει εδώ. Εξερευνήσαμε την απόδοση και επιτυχία των μεθόδων Nearest Neighbor, 3-Nearest-Neighbor και Nearest Centroid για να λύσουν ένα πρόβλημα κατηγοριοποίησης φωτογραφιών σε κλάσεις σύμφωνα με το περιεχόμενό τους. Έπειτα, δοκιμάσαμε διάφορους αλγόριθμους pre-processing ώστε να αυξηθεί η επιτυχία των μεθόδων και συγκρίναμε τα αποτελέσματα. Σιγά-σιγά θα προχωρήσουμε στην υλοποίηση ενός νευρωνικού επειδή αυτό λύνει προβλήματα τέτοιου είδους καλύτερα.